In [1]:
import json 

with open('判決書資料_keybert_filtered.json') as f:
    data = json.load(f)

In [1]:
from keybert import KeyBERT
from udicOpenData.stopwords import *
from transformers import BertTokenizerFast, pipeline, AutoModel
import torch
from tqdm.notebook import tqdm

tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
# ner = pipeline("ner", model="ckiplab/bert-base-chinese-ner", tokenizer = tokenizer, device = 0)
ner = pipeline("ner", model="xiaxy/elastic-bert-chinese-ner", tokenizer = tokenizer, device = 0)

def keyword_cleaner(keywords):
    new_keywords = keywords.copy()
    for keyword in keywords:
        results = ner(keyword)
        for res in results:
            if "GPE" in res["entity"] or "PERSON" in res["entity"] or "DATE" in res["entity"] or "TIME" in res["entity"]:
                new_keywords.remove(keyword)
                break
    return new_keywords
    
def deep_key_extraction(passage, topk=25):
    model = KeyBERT('paraphrase-multilingual-MiniLM-L12-v2')
    word_and_tag = list(rmsw(passage, flag=True))
    # print(word_and_tag)
    _temp = list(filter(lambda x: x[1]!='nr' and x[1]!='ng', word_and_tag))
    # print(_temp)
    # Extract keywords from a Chinese text
    text = " ".join(list(map(lambda x:x[0], _temp)))
    keywords = model.extract_keywords(text, keyphrase_ngram_range=(1,1), stop_words=None, use_mmr=True, use_maxsum=False, top_n=100)
    keywords = list(map(lambda x:x[0], keywords))
    keywords = keyword_cleaner(keywords[:topk])
    return keywords

ModuleNotFoundError: No module named 'keybert'

###### 一、李碧雲、李碧芳及李沈宏均為李錦樟之子女，詎李碧雲、李碧芳明知李沈宏已於民國103年7月6日因病死亡，權利能力業已終止，死亡後存款為全體繼承人之財產，須填寫繼承存款申請書經全部繼承人蓋章同意後始得提領，詎李碧雲、李碧芳共同基於行使偽造私文書、意圖為李錦樟不法所有之犯意聯絡，未得李沈宏全體法定繼承人之同意，於翌（7）日持李沈宏之印章、存摺至新竹縣湖口鄉農會，盜用李沈宏之印章蓋於取款憑條上，並持向不知情之農會辦事員徐翠雯虛偽表示李沈宏欲將該農會帳號：00000000000000號帳戶內現金新臺幣（下同）110萬1,400元匯款至李錦樟之該農會帳號：00000000000000號帳戶內而行使之，致徐翠雯不知李沈宏已死亡而陷於錯誤，依被告2人之申請辦理，使李錦樟取得前開金額，足生損害於李沈宏法定繼承人李忠桓、李松畇、李畋鵬繼承財產之權益，暨湖口鄉農會審核其會員金融帳戶存、提款申請之正確性及財產之損害。嗣經李忠桓於同年月15日至該農會查詢李沈宏帳戶交易紀錄，始悉上情。

In [ ]:
corpus_hashtags = []
for idx in tqdm(range(len(data))):
   res = deep_key_extraction(data[idx]["contents"])
   corpus_hashtags.append(res)

In [10]:
augmented_corpus = []
for i, c in enumerate(data):
    _corpus={}
    _corpus["id"]=i
    _corpus["content"]=data[i]["contents"]
    _corpus["hash_tags"]= " ".join(corpus_hashtags[i])
    augmented_corpus.append(_corpus)

In [11]:
Data = json.dumps(augmented_corpus, indent = 4, ensure_ascii=False)
with open("判決書資料_keybert_filtered.json", 'w', newline='\n', encoding = "utf-8") as f:
    f.write(Data)

#### pre - process

In [ ]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from tqdm.auto import tqdm

# 初始化BERT模型和tokenizer，并将它们放到GPU上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('huang0624/TW-ZH-LegalBERT-wwm').to(device)


# 将句子转换成dense embedding，并将它们放到CPU上
vecs = []
for sentence in tqdm(corpus_hashtags):
    tokens = tokenizer(sentence, max_length = 256, padding='max_length', truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model(**tokens)[0].cpu().numpy()
        vecs.append(output.reshape(-1))

vecs = np.vstack(vecs)

# 打印dense embedding的形状
print(f"Shape of dense embeddings: {vecs.shape}")
np.save('keybert_filtered.npy', vecs)

### Inference

In [3]:
import numpy as np 
vecs = np.load("keybert_filtered.npy")

In [4]:
import faiss
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from tqdm.auto import tqdm 

# 初始化BERT模型和tokenizer，并将它们放到GPU上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('huang0624/TW-ZH-LegalBERT-wwm').to(device)
# 要比較的句子
query = "一、李碧雲、李碧芳及李沈宏均為李錦樟之子女，詎李碧雲、李碧芳明知李沈宏已於民國103年7月6日因病死亡，權利能力業已終止，死亡後存款為全體繼承人之財產，須填寫繼承存款申請書經全部繼承人蓋章同意後始得提領，詎李碧雲、李碧芳共同基於行使偽造私文書、意圖為李錦樟不法所有之犯意聯絡，未得李沈宏全體法定繼承人之同意，於翌（7）日持李沈宏之印章、存摺至新竹縣湖口鄉農會，盜用李沈宏之印章蓋於取款憑條上，並持向不知情之農會辦事員徐翠雯虛偽表示李沈宏欲將該農會帳號：00000000000000號帳戶內現金新臺幣（下同）110萬1,400元匯款至李錦樟之該農會帳號：00000000000000號帳戶內而行使之，致徐翠雯不知李沈宏已死亡而陷於錯誤，依被告2人之申請辦理，使李錦樟取得前開金額，足生損害於李沈宏法定繼承人李忠桓、李松畇、李畋鵬繼承財產之權益，暨湖口鄉農會審核其會員金融帳戶存、提款申請之正確性及財產之損害。嗣經李忠桓於同年月15日至該農會查詢李沈宏帳戶交易紀錄，始悉上情。"
query_text = " ".join(deep_key_extraction(query))
print(query_text)
# 將要比較的句子轉換成dense embedding
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokens = tokenizer(query_text, max_length = 256, padding='max_length', truncation=True, return_tensors="pt").to(device)
with torch.no_grad():
    query_vec = model(**tokens)[0].cpu().numpy().reshape(-1)

# 計算query與向量庫中所有向量的相似度
index = faiss.IndexFlatIP(vecs.shape[1])
index.add(vecs)
distances, indices = index.search(np.array([query_vec]), k=5)

# 打印出相似度最高的前5筆資料
for i, index in enumerate(indices[0]):
    print(f"Rank {i+1}: {data[index]}, similarity score: {distances[0][i]}")
    # print(data[index]["hash_tags"], "\n\n")

Some weights of the model checkpoint at huang0624/TW-ZH-LegalBERT-wwm were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at huang0624/TW-ZH-LegalBERT-wwm and are newly initialized: ['bert.pooler.dense.weight', 'bert.poo

民國 繼承 繼承人 下同 表示 業已 新臺幣 文書 書經 權益 共同 存摺 子女 明知 取得 偽造 財產 犯意 法定
Rank 1: {'id': 877, 'content': '一、李碧雲、李碧芳及李沈宏均為李錦樟之子女，詎李碧雲、李碧芳明知李沈宏已於民國103年7月6日因病死亡，權利能力業已終止，死亡後存款為全體繼承人之財產，須填寫繼承存款申請書經全部繼承人蓋章同意後始得提領，詎李碧雲、李碧芳共同基於行使偽造私文書、意圖為李錦樟不法所有之犯意聯絡，未得李沈宏全體法定繼承人之同意，於翌（7）日持李沈宏之印章、存摺至新竹縣湖口鄉農會，盜用李沈宏之印章蓋於取款憑條上，並持向不知情之農會辦事員徐翠雯虛偽表示李沈宏欲將該農會帳號：00000000000000號帳戶內現金新臺幣（下同）110萬1,400元匯款至李錦樟之該農會帳號：00000000000000號帳戶內而行使之，致徐翠雯不知李沈宏已死亡而陷於錯誤，依被告2人之申請辦理，使李錦樟取得前開金額，足生損害於李沈宏法定繼承人李忠桓、李松畇、李畋鵬繼承財產之權益，暨湖口鄉農會審核其會員金融帳戶存、提款申請之正確性及財產之損害。嗣經李忠桓於同年月15日至該農會查詢李沈宏帳戶交易紀錄，始悉上情。', 'hash_tags': '民國 繼承 繼承人 下同 表示 業已 新臺幣 文書 書經 權益 共同 存摺 子女 明知 取得 偽造 財產 犯意 法定'}, similarity score: 108956.46875
Rank 2: {'id': 3546, 'content': '一、陳玉芬、陳秀玲明知其母曾美菊業於民國106年1月21日死亡，權利能力已消滅，不得為任何法律行為之主體，且曾美菊死亡，當時所有財產均變為遺產，其遺產為全體繼承人陳玉芬、陳秀玲、陳啟皜、陳啟緗等當然取得公同共有，陳玉芬、陳秀玲竟基於偽造私文書之犯意聯絡，未經全體繼承人之同意，即利用陳玉芬保管曾美菊所有台新國際商業銀行00000000000000號帳戶（下稱台新銀行帳戶）、遠東國際商業銀行新竹經國分行00000000000000號帳戶（下稱遠東銀行帳務）、元大商業銀行新竹分行00000000000000號帳戶（下稱元大銀行帳戶）之存摺、印章機會，於曾美菊死後之106年1月23日至同年月26日期間，盜蓋曾美菊之印鑑章於匯款憑條後持以行使，提領曾美菊所有之台